### 1.0 Imports

In [116]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import glob
import datetime as dt
from math import radians, cos, sin, asin, sqrt

### 2.0 Load Data

Loading in the cleaned, exported data from my data wrangling step.

In [117]:
#load trees data
trees_data = pd.read_csv('../data/data_outputs/seattle_trees.csv')

#load climate data
climate_data = pd.read_csv('../data/data_outputs/washington_climate_normals.csv')

### 3.0 Explore and Combine Data

The first step I have in mind is creating a field that maps each trees to its nearest weather station from my climate normals data.

#### 3.1 Tie Together The Two Dataframes Using Lat-Lon

##### 3.1.1 Define Functions

In [118]:
def dist_b_points(lat1, long1, lat2, long2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees). 
    Credit https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546
    """
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [119]:
def flag_nearest(df, lat, long, field):
    """ 
    Label the closest weather station using the reults of the dist_b_points function.
    Credit https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python-ce50d62af546
    """
    distances = df.apply(
        lambda row: dist_b_points(lat, long, row['lat'], row['long']), axis = 1)
    return df.loc[distances.idxmin(), str(field)]

##### 3.1.2 Rename Lat-Lons Fields

In [120]:
#Need a reminder of my fields
trees_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162080 entries, 0 to 162079
Data columns (total 10 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   planted_date               159111 non-null  object 
 1   most_recent_observation    162075 non-null  object 
 2   common_name                161408 non-null  object 
 3   longitude_coordinate       162080 non-null  float64
 4   latitude_coordinate        162080 non-null  float64
 5   diameter_breast_height_CM  162080 non-null  float64
 6   condition                  162080 non-null  object 
 7   native                     162080 non-null  object 
 8   age_at_obs                 159106 non-null  float64
 9   condition_index            162080 non-null  float64
dtypes: float64(5), object(5)
memory usage: 12.4+ MB


In [121]:
climate_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   STATION          33 non-null     object 
 1   LATITUDE         33 non-null     float64
 2   LONGITUDE        33 non-null     float64
 3   ELEVATION        33 non-null     float64
 4   NAME             33 non-null     object 
 5   ANN-TAVG-NORMAL  33 non-null     float64
 6   ANN-TMIN-NORMAL  33 non-null     float64
 7   ANN-TMAX-NORMAL  33 non-null     float64
 8   ANN-DUTR-NORMAL  33 non-null     float64
 9   ANN-PRCP-NORMAL  32 non-null     float64
dtypes: float64(8), object(2)
memory usage: 2.7+ KB


In [122]:
#Rename lat-long fields to easily identify between dfs
trees_data.rename(columns = {'longitude_coordinate': 'long_trees', 'latitude_coordinate': 'lat_trees'}, inplace=True)
climate_data.rename(columns = {'LONGITUDE': 'long_climate', 'LATITUDE': 'lat_climate'}, inplace=True)

##### 3.1.3 Assign Each Tree Record a Unique ID

I'm thinking in order to map back into our dataframe the closest station, I'll need to do it off a primary key and I'd rather not do that just off the location.

In [130]:

#Assign the key numerically, using the index number +1 (to exclude 0)
trees_data['tree_id'] = trees_data.index+1

In [131]:
#Double check values are unique
trees_data['tree_id'].value_counts()

1         1
108057    1
108050    1
108051    1
108052    1
         ..
54028     1
54029     1
54030     1
54031     1
162080    1
Name: tree_id, Length: 162080, dtype: int64

In [132]:
trees_data.head()

,planted_date,most_recent_observation,common_name,long_trees,lat_trees,diameter_breast_height_CM,condition,native,age_at_obs,condition_index,tree_id
0,1991-07-22,2019-04-27,(european) white birch,-122.282080,47.635207,40.64,excellent,introduced,27.765115,5.0,1
1,1990-08-16,2019-04-27,Red oak,-122.310243,47.597548,33.02,good,introduced,28.696003,4.0,2
2,1992-06-25,2019-04-27,Apple/crabapple,-122.355204,47.673916,15.24,dead/dying,no_info,26.836964,1.0,3
3,1991-07-30,2019-04-27,Kwanzan flowering cherry,-122.318952,47.649141,5.08,fair,no_info,27.743212,3.0,4
4,1991-07-16,2019-04-27,English (hedge) maple,-122.344731,47.613092,15.24,excellent,introduced,27.781542,5.0,5


##### 3.1.4 Merge Datasets Into a Single Table

In [126]:
trees_climate_all = trees_data[['lat_trees','long_trees']]

In [127]:
trees_climate_all['tree_id'] = trees_climate_all[['lat_trees','long_trees']].sum(axis=1).map(hash)

/var/folders/_z/9zltq2bj1dz1k05g4mtz4gq00000gn/T/ipykernel_1531/2365701839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trees_climate_all['tree_id'] = trees_climate_all[['lat_trees','long_trees']].sum(axis=1).map(hash)


In [128]:
trees_climate_all.head()

,lat_trees,long_trees,tree_id
0,47.635207,-122.282080,-1491587815483375690
1,47.597548,-122.310243,-1643362783451545674
2,47.673916,-122.355204,-1570942042198114378
3,47.649141,-122.318952,-1544479473013030986
4,47.613092,-122.344731,-1687043128503304266


In [129]:
#dist_b_points(trees_data['lat-trees'], trees_data['long-trees'], climate_data['lat-climate'], climate_data['long-climate'])
